In [1]:
import os, sys
import numpy as np

import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colors

from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
#
# Initialisation d'un cluster de 32 coeurs
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4, protocol = 'tcp')
client = Client(cluster)
client

/home/durandy/miniconda3/envs/towel/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46445 instead
  warnings.warn(
/home/durandy/miniconda3/envs/towel/lib/python3.9/contextlib.py:126: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46445/status,
Dashboard: http://127.0.0.1:46445/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: tcp://127.0.0.1:33797,Workers: 1
Dashboard: http://127.0.0.1:46445/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://127.0.0.1:38591,Total threads: 4
Dashboard: http://127.0.0.1:39863/status,Memory: 78.61 GiB
Nanny: None,


In [2]:
def isLeapYear (yearN):
    if ((yearN % 4 == 0) and (yearN % 100 != 0)) or (yearN % 400 == 0):
        reponse = True
    else:
        reponse = False
    print(reponse, '\n')
    return reponse

def hour_mean(x):
     return x.groupby('time.hour').mean('time')
    
def hour_sum(x):
     return x.groupby('time.hour').sum('time')
    
def hour_std(x):
     return x.groupby('time.hour').std('time')
    
def on1Dtime(ds,v):
    attrs = {"units": "hours since 1900-01-01", "calendar":"gregorian"}
    ds_T = xr.Dataset({"time": ("time", np.arange(4*365*24,4*365*24+366*24,3), attrs)})

    #### Reconstruction of the data
    da = xr.DataArray(
        data = np.reshape(ds[v].values,(np.shape(ds.dayofyear)[0]*np.shape(ds.hour)[0],np.shape(ds.level)[0],np.shape(ds.latitude)[0],np.shape(ds.longitude)[0])),
        dims = ["time","level","latitude","longitude"],
        coords=dict(
        time = ds_T.time,
        level = ds.level,
        latitude = ds.latitude,
        longitude = ds.longitude
        ),
        attrs=dict(
        description="vo_clim",
        units="s-1",
        ),
    )
    ds = da.to_dataset(name = v)
    return ds


In [8]:
indir = '/cnrm/tropics/commun/DATACOMMUN/CERES/SYN1deg/CERES_SYN1deg-1H_Terra-Aqua-MODIS_Ed'
outdir = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_CLIM/'

ds = xr.open_mfdataset(indir + '*.nc', chunks = {'time' : 100}, parallel=True)

# # group by dayofyear, then apply the function:
# ds_clim = ds.groupby('time.dayofyear').apply(hour_mean).compute()
# ds_clim.to_netcdf(outdir + 'clim_tcwv_brut_ERA5_3H_1979_2020.nc')

In [6]:
nbYear = 1
minYear = 1990
maxYear = 2020
year = np.arange(minYear,maxYear,nbYear)
datasets = []

for y in year :
    ds = xr.open_dataset(indir+exp+str(y)+'.nc', chunks = {'time' : 1}
                        )
    datasets.append(ds)
        
ds = xr.concat(datasets, dim='time', coords='minimal', compat='override')

[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019]
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [9]:
_ds = ds['toa_lw_all_1h']
_ds = _ds.isel(time = slice(0,None,3))
_ds

<xarray.DataArray 'toa_lw_all_1h' (time: 59664, lat: 60, lon: 360)>
dask.array<getitem, shape=(59664, 60, 360), dtype=float32, chunksize=(34, 60, 360), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-03-01T00:30:00.000053644 ... 2020-07-...
  * lon      (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * lat      (lat) float32 -29.5 -28.5 -27.5 -26.5 -25.5 ... 26.5 27.5 28.5 29.5
Attributes:
    long_name:      Observed Top of the Atmosphere Longwave Flux, All-sky con...
    standard_name:  Observed TOA Longwave Flux - All-sky
    units:          W m-2
    valid_min:             0
    valid_max:           500

In [11]:
time_R = pd.date_range(start="2000-03-01", end="2020-07-31T21", freq = '3H')
time_R

DatetimeIndex(['2000-03-01 00:00:00', '2000-03-01 03:00:00',
               '2000-03-01 06:00:00', '2000-03-01 09:00:00',
               '2000-03-01 12:00:00', '2000-03-01 15:00:00',
               '2000-03-01 18:00:00', '2000-03-01 21:00:00',
               '2000-03-02 00:00:00', '2000-03-02 03:00:00',
               ...
               '2020-07-30 18:00:00', '2020-07-30 21:00:00',
               '2020-07-31 00:00:00', '2020-07-31 03:00:00',
               '2020-07-31 06:00:00', '2020-07-31 09:00:00',
               '2020-07-31 12:00:00', '2020-07-31 15:00:00',
               '2020-07-31 18:00:00', '2020-07-31 21:00:00'],
              dtype='datetime64[ns]', length=59664, freq='3H')

In [12]:
_ds['time'] = time_R

In [14]:
ds = _ds.to_dataset(name = 'toa_lw_all_1h')
ds

<xarray.Dataset>
Dimensions:        (time: 59664, lon: 360, lat: 60)
Coordinates:
  * time           (time) datetime64[ns] 2000-03-01 ... 2020-07-31T21:00:00
  * lon            (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat            (lat) float32 -29.5 -28.5 -27.5 -26.5 ... 26.5 27.5 28.5 29.5
Data variables:
    toa_lw_all_1h  (time, lat, lon) float32 dask.array<chunksize=(34, 60, 360), meta=np.ndarray>

In [15]:
ds.to_netcdf('/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/RAW_CLIM/OLR.nc')